In [1]:
import math
import time
import csv
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import os
from os import path
from functions import *
from exceptions import *
#import lmfit


In [2]:
#####################################################################################################################
# Give all filepaths to a directory containing "resets_output.txt" and "g4_output.txt" for a set of events saved as a dataframe.
# Each filepath must be given as an element of the list, filepaths, even if it is only a single filepath
#####################################################################################################################


filepaths = ['../SampleData/', #enter filepath for first data set here
             '', #enter filepath for second data set here
             '', #enter filepath for third data set here
            ]


#####################################################################################################################
# This will assign the following values to each unique active pixel in each event of each filepath given above
#   - Mean_TOA (Mean Time of Arrival of all resets in pixel readout)
#   - StDev (Simple Standard Deviation of all resets in pixel readout)
#   - z_avg (Average Z of deposits in the general vicinity of the pixel's X-Y projection)
#
# At the end of each file, it will save the new unique_active_pixels.txt over the original
#####################################################################################################################

for filepath in filepaths:

    print("Next file: %s"%filepath)

    if os.path.exists(str('%sresets_output.txt'%filepath)) == False:
        raise DataNotPresentError 
    sim_data = os.path.exists(str('%sg4_output.txt'%filepath))
    if sim_data == True:
        print('g4_output.txt was found in the given directory... Proceeding as simulated data.')
    else:
        print('g4_output.txt was not found in the given directory... Proceeding as real data.')
    
    
    # Load reset data as resets and g4 data as g4
    print('Loading dataframes...')
    print('--> resets...')
    resets = pd.read_csv('%sresets_output.txt'%filepath,index_col=0)
    if sim_data == True:
        print('--> g4...')
        g4 = pd.read_csv('%sg4_output.txt'%filepath,index_col=0)

        if resets.event.min() != g4.event.min() or resets.event.max() != g4.event.max():
            raise IndexError('Event numbers in resets and g4 do not correlate')
        events = np.arange(int(resets.event.min()),int(resets.event.max())+1,1)

    unique_active_pixels = resets.groupby(['event','pixel_x','pixel_y']).size().reset_index().rename(columns={0:'',1:'event',2:'pixel_x',3:'pixel_y',4:'nResets'})
    unique_active_pixels = unique_active_pixels.rename(columns={'event':'event','pixel_x':'pixel_x','pixel_y':'pixel_y','Unnamed: 4':'nResets'})

    
    
    old_event = -1
    for idx,row in unique_active_pixels.iterrows():
        if row.event != old_event:
            print('Processing Event %d'%row.event)
            old_event = row.event
        tmp = resets[(resets.event == row.event) & (resets.pixel_x == row.pixel_x) & (resets.pixel_y == row.pixel_y)]
        Mean_TOA = tmp.reset_time.mean()
        StDev = tmp.reset_time.std()
        unique_active_pixels.at[idx,'Mean_TOA'] = Mean_TOA
        unique_active_pixels.at[idx,'StDev'] = StDev
        start,stop = math.floor(tmp.reset_time.min()*1e+07)/1e+07,math.ceil(tmp.reset_time.max()*1e+07)/1e+07
        if math.isnan(tmp.reset_time.std()) == False and math.isnan(tmp.reset_time.mean()) == False: 
            start,stop = math.floor((Mean_TOA - 3*StDev)*1e+07)/1e+07,math.ceil((Mean_TOA + 3*StDev)*1e+07)/1e+07
            if start > tmp.reset_time.min():
                start = math.floor(tmp.reset_time.min()*1e+07-5)/1e+07
            if stop < tmp.reset_time.max():
                start = math.ceil(tmp.reset_time.max()*1e+07+5)/1e+07 
        
        
        x_min,x_max = (row['pixel_x']-2)*4/10,(row['pixel_x']+1)*4/10
        y_min,y_max = (row['pixel_y']-2)*4/10,(row['pixel_y']+1)*4/10
        
        if sim_data == True:
            g4_pix = g4[(g4.event == row['event']) & (g4.xi <= x_max) & (g4.xf >= x_min) & (g4.yi <= y_max) & (g4.yf >= y_min)]
            if len(g4_pix.index) == 0:
                g4_pix = g4[(g4.event == row['event']) & (g4.xi <= x_max+4/10) & (g4.xf >= x_min-4/10) & (g4.yi <= y_max+4/10) & (g4.yf >= y_min-4/10)]
            #print("length of g4_pix:",len(g4_pix.index))
            unique_active_pixels.at[idx,'z_avg'] = (g4_pix.zi.sum() + g4_pix.zf.sum())/(2*len(g4_pix.index))

    unique_active_pixels = unique_active_pixels.rename(columns={"event":"event","pixel_x":"pixel_x","pixel_y":"pixel_y","":"nResets","Mean_TOA":"Mean_TOA","StDev":"StDev","z_avg":"z_avg"}) 
    print(unique_active_pixels)
    print('--> Saving all events to file...'%old_event, end="")
    unique_active_pixels.to_csv('%sUniqueActivePixels.txt'%filepath)
    print('   Done!')
    

Next file: ../SampleData/
g4_output.txt was found in the given directory... Proceeding as simulated data.
Loading dataframes...
--> resets...
--> g4...
Processing Event 0
Processing Event 1
Processing Event 2
Processing Event 3
Processing Event 4
Processing Event 5
Processing Event 6
Processing Event 7
Processing Event 8
Processing Event 9
       event  pixel_x  pixel_y  nResets  Mean_TOA         StDev       z_avg
0          0      152      883        2  0.001578  6.576093e-07  259.879643
1          0      152      884        4  0.001579  1.878208e-06  259.912136
2          0      152      885        1  0.001579           NaN  260.178696
3          0      153      882        3  0.001577  8.507056e-07  259.873947
4          0      153      883       10  0.001578  2.156110e-06  260.307858
...      ...      ...      ...      ...       ...           ...         ...
45422      9      347      985        1  0.001786           NaN  294.162083
45423      9      348     1087        1  0.001607 

DataNotPresentError: No reset data file (resets_output.txt) is present in the given directory